In [ ]:
import sys
sys.path.append('/home/onyxia/work/libsigma')
import numpy as np
import geopandas as gpd
from osgeo import gdal, ogr
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
import classification as cla
import os

# define parameters
shp_path = '../results/data/sample/Sample_BD_foret_T31TCJ.shp'
raster_path = '../results/data/img_pretraitees/Serie_temp_S2_ndvi.tif'
raster_temp_path = '../results/data/classif/raster_temp.tif'
output_path = '../results/data/classif/carte_essences_echelle_pixel.tif'
field_name = 'Code_pixel'  # field containing the numeric label of the classes

In [2]:
# Calculer les coordonnées xmin, ymin, xmax, ymax
spatial_resolution = 10
xmin = 501135.41
ymin = 6240660.64
xmax = 609755.41
ymax = 6314460.64

# Maintenant, utilisez ces coordonnées dans la commande gdal_rasterize
cmd_pattern = ("gdal_rasterize -a {field_name} "
                "-tr {spatial_resolution} {spatial_resolution} "
                "-te {xmin} {ymin} {xmax} {ymax} -ot Byte -of GTiff "
                "{in_vector} {out_image}")

# Remplir la commande avec les paramètres
cmd = cmd_pattern.format(in_vector=shp_path, xmin=xmin, ymin=ymin, xmax=xmax,
                        ymax=ymax, out_image=raster_temp_path, field_name=field_name,
                        spatial_resolution=spatial_resolution)
                    
os.system(cmd)

0...10...20...30...40...50...60...70...80...90...100 - done.


0

In [3]:
X, Y, t = cla.get_samples_from_roi(raster_path, raster_temp_path)
print(X.shape)
print(Y.shape)
# split samples
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5)

/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


(2468375, 60)
(2468375, 1)


In [4]:
rf_classifier = RandomForestClassifier(
    max_depth=50,          # Profondeur maximale de l'arbre
    oob_score=True,        # Utilisation de l'échantillonnage en dehors de la sac (Out-of-Bag)
    max_samples=0.75,      # Utilisation de 75% des échantillons pour chaque arbre
    class_weight='balanced',  # Rééquilibrage des classes
)

# Entraîner le modèle sur les données d'entraînement
rf_classifier.fit(X_train, Y_train.ravel())

# Prédire les étiquettes pour l'ensemble de test
Y_pred = rf_classifier.predict(X_test)

# Évaluer la performance du modèle
accuracy = accuracy_score(Y_test, Y_pred)
print(f"Précision du modèle : {accuracy:.2f}")

# Afficher le rapport de classification
print("\nRapport de classification :")
print(classification_report(Y_test, Y_pred))

# Afficher le score OOB (out-of-bag)
print(f"\nScore OOB : {rf_classifier.oob_score_:.2f}")

Précision du modèle : 0.93

Rapport de classification :
              precision    recall  f1-score   support

          11       0.98      0.68      0.80     35302
          12       0.97      0.96      0.96   1112795
          13       0.92      0.41      0.56     11012
          14       0.28      0.69      0.40     27027
          21       0.99      0.68      0.81       721
          22       0.98      0.71      0.82      1945
          23       0.97      0.86      0.91     10815
          24       0.93      0.77      0.85     26178
          25       0.96      0.75      0.84      8393

    accuracy                           0.93   1234188
   macro avg       0.89      0.72      0.77   1234188
weighted avg       0.95      0.93      0.94   1234188


Score OOB : 0.93


In [5]:
# Définir les paramètres de validation croisée répétée
n_splits = 5
n_repeats = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True)

# Variables pour stocker les performances globales
global_accuracies = []
global_f1_scores = []

# Boucle sur les répétitions
for repeat in range(n_repeats):
    print(f"Répétition {repeat + 1}/{n_repeats}")
    
    fold_accuracies = []
    fold_f1_scores = []

    # Boucle sur les splits de validation croisée
    for train_index, test_index in skf.split(X, Y):
        # Séparation des données en train/test
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        
        # Aplatir Y_train et Y_test en tableaux 1D
        Y_train = Y_train.ravel()
        Y_test = Y_test.ravel()
        
        # Entraînement du modèle
        rf_classifier.fit(X_train, Y_train)
        
        # Prédiction
        Y_pred = rf_classifier.predict(X_test)
        
        # Calcul des métriques pour chaque fold
        fold_accuracies.append(accuracy_score(Y_test, Y_pred))
        
        # Calcul des F1-scores par classe pour ce fold
        report = classification_report(Y_test, Y_pred, output_dict=True, zero_division=1)
        fold_f1_scores.append({
            label: report[label]['f1-score']
            for label in report if label not in ['accuracy', 'macro avg', 'weighted avg']
        })

    # Stockage des performances moyennes pour cette répétition
    global_accuracies.append(np.mean(fold_accuracies))
    
    # Moyenne des F1-scores pour chaque classe sur les folds
    avg_f1_scores = {label: np.mean([fold[label] for fold in fold_f1_scores])
                     for label in fold_f1_scores[0]}
    global_f1_scores.append(avg_f1_scores)

# Calcul des performances globales sur toutes les répétitions
mean_accuracy = np.mean(global_accuracies)
std_accuracy = np.std(global_accuracies)

# Moyenne des F1-scores par classe sur toutes les répétitions
mean_f1_scores = {label: np.mean([rep[label] for rep in global_f1_scores])
                  for label in global_f1_scores[0]}

# Affichage des résultats globaux
print(f"Précision moyenne : {mean_accuracy:.2f} ± {std_accuracy:.2f}")
print("\nF1-scores moyens par classe :")
for class_label, f1_score in mean_f1_scores.items():
    print(f"Classe {class_label} : {f1_score:.2f}")

Répétition 1/10


KeyboardInterrupt: 

In [ ]:
# Étape 1 : Entraîner le modèle sur tous les échantillons disponibles
rf_classifier.fit(X, Y)

# Étape 2 : Charger l'image raster et préparer les données pour prédiction
raster = gdal.Open(raster_path)
cols = raster.RasterXSize
rows = raster.RasterYSize
bands = raster.RasterCount
geo_transform = raster.GetGeoTransform()
projection = raster.GetProjection()

# Récupérer la valeur NoData du raster
band = raster.GetRasterBand(1)
nodata_value = band.GetNoDataValue()  # Récupérer la valeur NoData spécifiée

# Lire toutes les bandes en tant que tableau
raster_array = raster.ReadAsArray().astype(np.float32)  # Assurez-vous que les valeurs sont bien des float

# Identifier les zones vides (NoData) dans le raster original
nodata_mask = np.any(raster_array == nodata_value, axis=0)  # Exemple si 0 est utilisé comme NoData dans les bandes originales

# Réorganiser les dimensions pour avoir (nb_pixels, nb_bandes)
data_for_prediction = raster_array.reshape(bands, -1).T

# Étape 3 : Prédire les classes pour chaque pixel
predicted_classes = rf_classifier.predict(data_for_prediction)

# Reshape des prédictions pour correspondre à l'image d'origine
predicted_classes = predicted_classes.reshape(rows, cols)

# Étape 4 : Appliquer le masque nodata
predicted_classes[nodata_mask] = 0  # Forcer les zones vides à 0 (nodata)

# Étape 5 : Écrire la carte des essences en sortie
output_path = '../results/data/classif/carte_essences_echelle_pixel.tif'
driver = gdal.GetDriverByName('GTiff')
output_raster = driver.Create(output_path, cols, rows, 1, gdal.GDT_Byte)

# Définir les propriétés spatiales
output_raster.SetGeoTransform(geo_transform)
output_raster.SetProjection(projection)

# Ajouter la bande et écrire les données prédictes
output_band = output_raster.GetRasterBand(1)
output_band.WriteArray(predicted_classes)

# Fixer la valeur de nodata à 0
output_band.SetNoDataValue(0)

# Fermer le fichier
output_band.FlushCache()
output_raster = None

print(f"Carte des essences enregistrée sous : {output_path}")

/opt/conda/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
